In [1]:
import pandas as pd
import numpy as np
from elo_rating import EloRating
import mysql.connector

Connect to sql database to obtain tables.

In [2]:
def mysql_db_connect():
    host = input("\nHost Name: ")
    user = input("\nUser Name: ")
    passwd = input("\nPassword: ")
    database =  input("\nDatabase Name: ")
    try:
        db_connection = mysql.connector.connect(
            host=host,
            user=user,
            passwd=passwd,
            database=database
        )
    except:
        raise Exception
    else:
        print(f"CONNECTION TO: {host} ESTABLISHED")
        return db_connection

In [3]:
con = mysql_db_connect()


Host Name: localhost

User Name: root

Password: MySQL117!

Database Name: techscore
CONNECTION TO: localhost ESTABLISHED


Read the races and regattas tables into pandas dataframes.

In [4]:
try:
    cursor = con.cursor(buffered=True)
    races_sql = "SELECT * FROM races"
    regattas_sql = "SELECT * FROM regattas"
    sailors_sql = "SELECT * FROM sailors"
    races_df = pd.read_sql(races_sql, con)
    regattas_df = pd.read_sql(regattas_sql, con) 
    sailors_df = pd.read_sql(sailors_sql,con)
except:
    con.close()
    raise Exception
else:
    cursor.close()
    con.close()

Merge the regatta_name and start_date columns in races and regattas to get a unique identifier for the regatta (some regattas share the same name every year). This merge is the same as the composite primary key for the table.

In [5]:
regattas_df["regatta_unique"] = regattas_df["regatta_name"] + regattas_df["start_date"]
races_df["regatta_unique"] = races_df["regatta_name"] + races_df["start_date"]

We can see that the finishing place is an object in the below dtypes method result. We need finishing place to be ordered so that we can compare scores and determine winenrs and losers. Thus all character digits will be converted to int, and if the finishing place is DSQ, DNF, DNS, etc., we will convert it to the number of teams plus 1 (as per ICSA rules).

In [6]:
races_df.dtypes

regatta_name       object
start_date         object
sailor_name        object
team_name          object
subteam_name       object
division           object
race_number         int64
finishing_place    object
regatta_unique     object
dtype: object

We must go to the regatta in the regattas table and obtain the number of teams in the regatta. Then, when the finishing place throws an error for int('DSQ'), we will reassign 'DSQ' to the number of team plus one.

In [7]:
for j, sailor_row in races_df.iterrows():
    regatta_name = sailor_row['regatta_unique']
    num_teams = regattas_df.loc[regattas_df['regatta_unique']==regatta_name].num_teams.values[0]
    try:
        races_df.at[j, 'finishing_place'] = int(sailor_row['finishing_place'])
    except ValueError:
        races_df.at[j, 'finishing_place'] = num_teams + 1
    except:
        raise Exception

We will iterate through the regattas table chronologically and handle each race and division in each regatta separately. First, we must sort the regattas by date.

In [8]:
regattas_df['start_date'] = pd.to_datetime(regattas_df.start_date)
regattas_df = regattas_df.sort_values('start_date')

The regattas table is now in chronological order. The next step is to clean the races data, which contains KEY_ERROR and NOT_LISTED in some rows.

In [9]:
races_df_cleaned = races_df.loc[~((races_df['team_name']=='KEY_ERROR') | (races_df['sailor_name']=='KEY_ERROR') | (races_df['sailor_name']=="NOT_LISTED")), :]

Below we initialize the EloRating object defined in the elo_rating module.

In [10]:
sailor_ratings = EloRating()

Now we will go through all regattas and rank the sailors. 

In [11]:
sailor_ratings.clear_rankings()

In [12]:
for i, row in regattas_df.iterrows(): #iterrows() returns a generator.
    num_races = row['num_races_each_div']
    regatta_name = row['regatta_unique']
    num_divisions = int(row['divisions'].split(' ')[0]) #get number of divisions
    division_sequence = [chr(ascii_val) for ascii_val in range(65, 65 + num_divisions)] #sequence of capital letters
    races_at_regatta_df = races_df_cleaned.loc[races_df['regatta_unique']==regatta_name, :]
    
    for division in division_sequence: #iterate over all divisions (max 3)
        races_at_regatta_div_df = races_at_regatta_df.loc[races_at_regatta_df['division']==division, :]
        
        for race in range(1, num_races + 1): #iterate over all races (max approximately 20)
            races_at_regatta_div_race_df = races_at_regatta_div_df.loc[races_at_regatta_div_df['race_number']==race, :]
            races_at_regatta_div_race_df = races_at_regatta_div_race_df.sort_values('finishing_place')
            
            for j, sailor_row in races_at_regatta_div_race_df.iterrows():
                finishing_place = sailor_row['finishing_place']
                sailors_behind_df = races_at_regatta_div_race_df.loc[(races_at_regatta_div_race_df['finishing_place'] > finishing_place), :]
                
                for k, sailor_behind in sailors_behind_df.iterrows():
                    
                    faceoff_result = sailor_ratings.faceoff(sailor_row['sailor_name'], sailor_behind['sailor_name'], finishing_place, sailor_behind['finishing_place'], itemA_association = sailor_row['team_name'], itemB_association = sailor_behind['team_name'])
                    for key, val in faceoff_result.items(): sailor_ratings.modify_rating(key, val)  #update elos   

In [13]:
rankings = sailor_ratings.sort_copy(reverse=True)

Sort in descending order and export results to csv. 

In [14]:
sailor_ratings.sort(reverse=True)

In [15]:
sailor_ratings.rankings_to_csv("/home/jonathan/Documents/Py_Projects/Techscore_Proj/ratings.csv")

Now we can convert the output dictionary, modify the name in the sailors table of the database to be a combination of name and team name, and perform a left join between. This will allow us to perform sql queries on the rankings, or keep it in pandas form and query it.

First, we will make a dictionary where the keys are column names and the values are lists for those columns. Then, we will convert it to a dataframe using the from_dict() method in pandas.

In [17]:
dict_for_df = {"Rank" : [], "Name_Team" : [], "Elo_Rating" : [], "Nos_Races" : []}
for rank, (name, val) in enumerate(rankings.items(), 1):
    dict_for_df["Rank"].append(rank)
    dict_for_df["Name_Team"].append(name)
    dict_for_df["Elo_Rating"].append(val[0])  
    dict_for_df["Nos_Races"].append(val[1])

In [18]:
rankings_df = pd.DataFrame.from_dict(dict_for_df, orient="columns")

Combine the name and team in the sailors table.

In [19]:
sailors_df["sailor_unique"] = sailors_df["sailor_name"] + '_' + sailors_df["sailor_team_2"]

Left join with the sailors dataframe.

In [20]:
sailors_rankings_left_df = pd.merge(rankings_df, sailors_df, left_on = "Name_Team", right_on = "sailor_unique", how='left')

The left join result will have some null rows, as there is not a match in the sailors table. This is because some sailors, especially those with earlier graduation dates, do not have sailor pages on techscore, and the sailors table consists of strictly sailors with a sailor page. Thus, an inner join would represent keys that are present in both the sailors and rankings tables, which forms a complete table for analysis.

In [21]:
sailors_rankings_inner_df = pd.merge(rankings_df, sailors_df, left_on = "Name_Team", right_on = "sailor_unique", how='inner') 

Find subets according to your interests.

In [38]:
sailors_rankings_filter = sailors_rankings_inner_df.loc[(sailors_rankings_inner_df['Nos_Races']>700) & (sailors_rankings_inner_df['grad_year']>2018)]